# Step 1
# Image Acquisition 

In [1]:

#Image Acquisition 
import cv2
# Option 1: Acquire from webcam (real-time)
cap = cv2.VideoCapture(0)  # 0 = default camera
ret, frame = cap.read()    # Capture a frame
if ret:
    cv2.imwrite('acquired_image.jpg', frame)  # Save for next steps
cap.release()
# Option 2: Load from file (simulated acquisition)
# frame = cv2.imread('input.jpg')  # Use this if no camera
cv2.imshow('Acquired Image', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()



# Step 2
# Image Preprocessing


In [2]:
import cv2
img = cv2.imread('acquired_image.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Grayscale
blur = cv2.GaussianBlur(gray, (5,5), 0)       # Denoise
equalized = cv2.equalizeHist(blur)            # Contrast enhancement
cv2.imshow('Preprocessed', equalized)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('preprocessed.jpg', equalized)    # Save for next


True

# Step 3

In [3]:
import cv2
img = cv2.imread('preprocessed.jpg', 0)
_, thresh = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)  # Binary segmentation
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
output = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
cv2.drawContours(output, contours, -1, (0,255,0), 2)  # Draw segmented regions

cv2.imshow('Segmented', output)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('segmented.jpg', output)


True

# Step 4

In [4]:
import cv2
img = cv2.imread('segmented.jpg', 0)
orb = cv2.ORB_create()
keypoints, descriptors = orb.detectAndCompute(img, None)  # Extract features
output = cv2.drawKeypoints(img, keypoints, None, color=(0,255,0))
cv2.imshow('Features', output)
cv2.waitKey(0)
cv2.destroyAllWindows()
# Descriptors are arrays; save keypoints if needed


# Step 5

In [5]:
import cv2
import numpy as np

# Read images in grayscale
img1 = cv2.imread('segmented.jpg', 0)
img2 = cv2.imread('acquired_image.jpg', 0)

if img1 is None or img2 is None:
    print("Error: Check image paths")
    exit()

# Improve contrast (helps ORB detect features)
img1 = cv2.equalizeHist(img1)
img2 = cv2.equalizeHist(img2)

# ORB detector
orb = cv2.ORB_create(nfeatures=1500)

kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)

print("Keypoints1:", len(kp1))
print("Keypoints2:", len(kp2))

# Prevent crash if no descriptors found
if des1 is None or des2 is None:
    print("Error: No features detected in one of the images")
    exit()

# Ensure correct datatype
des1 = np.uint8(des1)
des2 = np.uint8(des2)

# Matcher (same method as yours)
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1, des2)

if len(matches) == 0:
    print("No matches found")
    exit()

# Sort matches by quality
matches = sorted(matches, key=lambda x: x.distance)

# Draw best matches
output = cv2.drawMatches(img1, kp1, img2, kp2, matches[:10], None)

cv2.imshow('Matches', output)
cv2.waitKey(0)
cv2.destroyAllWindows()


Keypoints1: 1500
Keypoints2: 1500


# Step 6

In [6]:
import cv2

img = cv2.imread('segmented.jpg', 0)
contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
output = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
for cnt in contours:
    area = cv2.contourArea(cnt)
    label = "Large" if area > 1000 else "Small"  # Simple classification
    cv2.drawContours(output, [cnt], 0, (0,255,0), 2)
    cv2.putText(output, label, (cnt[0][0][0], cnt[0][0][1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1)
cv2.imshow('Classified', output)
cv2.waitKey(0)
cv2.destroyAllWindows()


# Step 7

In [7]:
import cv2
import numpy as np

# Example detections → [x, y, w, h, confidence]
detections = np.array([
    [100, 100, 50, 50, 0.9],
    [105, 105, 50, 50, 0.8]
])

boxes = detections[:, :4].tolist()
scores = detections[:, 4].tolist()

indices = cv2.dnn.NMSBoxes(boxes, scores, score_threshold=0.5, nms_threshold=0.4)

img = cv2.imread('segmented.jpg')
if img is None:
    print("Image not found")
    exit()

if len(indices) > 0:
    for i in indices.flatten():  # FIX
        x, y, w, h = detections[i, :4].astype(int)
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

print("Detected objects:", len(indices))

cv2.imshow('Post-processed Output', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


Detected objects: 1


# Step 8

In [ ]:
import cv2

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade  = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

cap = cv2.VideoCapture(0)  # Webcam

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 1)

    cv2.imshow('Classical Face + Eye Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
